<a href="https://colab.research.google.com/github/GGYYZZ/My-test/blob/main/%E2%80%9CBasic_Algorithm_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 查看分配到的设备
# 查看设备，是K80还是T4,如果是K80...！
! /opt/bin/nvidia-smi

/bin/bash: /opt/bin/nvidia-smi: No such file or directory


In [ ]:
#@title 挂载谷歌云盘
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My Drive/LBSN/dataset1

Mounted at /content/drive
/content/drive/My Drive/LBSN/dataset1


In [ ]:
# %load Basic Algorithm.py
import networkx as nx
import timeit

# 全局变量
# 用户网络GU 位置节点群GL(点属性xco,yco) 签入网络GC(边属性weight)
# GUQuerySet查询用户节点 GLQuerySet查询位置节点
# ResultSet结果集 ResultDegree结果集的平均签入密度
# 数据集文件user_network spot_location visited_spots
GU = nx.Graph()
GL = nx.Graph()
GC = nx.Graph()
GUQuerySet = set()
GLQuerySet = set()
ResultSet = set()
ResultDegree = 0.0


# 读取文件生成GU
def load_GU():
    with open("/content/drive/My Drive/LBSN/dataset1/user_network.txt", "r") as file:
    # with open("./dataset1/user_network.txt", "r") as file:
        start = ""
        flag = 1
        for line in file:
            tmp1 = line.split("]")
            tmp1.pop()                             # 删去每行最后一个换行符"\n"
            for x in tmp1:
                tmp2 = x.split("[")
                for y in tmp2:
                    lis = y.split(",")
                    if flag:                       # 画图，点---边---点......
                        GU.add_node('U'+lis[0])
                        start = 'U'+lis[0]
                        flag = 0
                    else:
                        for v in lis:
                            GU.add_node('U'+v)
                            GU.add_edge(start, 'U'+v)
                        flag = 1


# 读取文件生成GC
def load_GC():
    global GC
    with open("/content/drive/My Drive/LBSN/dataset1/visited_spots.txt", "r") as file:
    # with open("./dataset1/visited_spots.txt", "r") as file:
        start = ""
        flag = 1
        for line in file:
            tmp1 = line.split("]")
            tmp1.pop()
            for x in tmp1:
                tmp2 = x.split("[")
                for y in tmp2:
                    lis = y.split(",")
                    # print(lis)
                    if flag:
                        GC.add_node('U'+lis[0])
                        start = 'U'+lis[0]
                        flag = 0
                    else:
                        for x in range(0, len(lis)-1, 2):
                            GC.add_node('L'+lis[x])
                            GC.add_edge(start, 'L'+lis[x], weight=lis[x+1])
                            # 定义6（签入权重）。签入权重W(u, v）是签入图GC中的边E(u, v)
                            # 的属性，其中E(u, v)∈EC。签入权重的值可以用签入的频率、滞留时间以及近期性等作为评价标准。
                        flag = 1
    # print(GC.number_of_nodes())
    # print(list(GC.nodes))
    # print(GC.number_of_edges())
    # print(list(GC.edges))
    # for x in list(GC.edges):
    #     print(x, GC.edges[x]['weight'])


# 判断两点间是否在距离阈值内
def is_reachable(v1, v2, g):
    global r
    x1 = float(g.nodes[v1]['xco'])
    y1 = float(g.nodes[v1]['yco'])
    x2 = float(g.nodes[v2]['xco'])
    y2 = float(g.nodes[v2]['yco'])
    if (x1-x2)**2 + (y1-y2)**2 > r**2:
        return False
    return True


# 读取文件生成GL(在距离阈值内的图)
def load_GL():
    with open("/content/drive/My Drive/LBSN/dataset1/spot_location.txt", "r") as file:
    # with open("./dataset1/spot_location.txt", "r") as file:
        for line in file:
            lis = line.split()
            for x in range(0, len(lis), 3):
                GL.add_node('L'+lis[x], xco=lis[x + 1], yco=lis[x + 2], rflag=0)
    for i in list(GL.nodes):
        GL.nodes[i]["rflag"] = 1
        for j in list(GL.nodes):
            if GL.nodes[j]["rflag"] == 0:
                if is_reachable(i, j, GL):
                    # print(i, j)
                    GL.add_edge(i, j)
    # print(GL.number_of_nodes())
    # print(list(GL.nodes))
    # for x in list(GL.nodes):
    #     print(x, GL.nodes[x]['xco'], GL.nodes[x]['yco'])
    # print(GL.number_of_edges())
    # print(list(GL.edges))


# 获取包含查询节点的CBCC
# gflag为1则查询用户图，为0则查询位置图
def get_CBCC(k, g, gflag):
    Gtmp = k_Core(k, g)
    if Gtmp:
        if gflag == 1:
            if judge_CBCC(Gtmp, GUQuerySet):
                return True
        else:
            if judge_CBCC(Gtmp, GLQuerySet):
                return True
    return False


# 用k-core算法获得CBCC
def k_Core(k, g):
    flag = 0
    for i in list(g.nodes):
         if g.degree[i] < k:
             g.remove_node(i)
             flag = 1
    if flag:
        return k_Core(k, g)
    else:
        return g


# 判断CBCC中是否包含所有的查询节点
def judge_CBCC(g, queryset):
    for i in queryset:
        if i not in list(g.nodes):
            return False
    return True


# 计算两个CBCC的平均签入密度
def get_Checkin(gu, gl, gc):
    global ResultDegree, ResultSet
    sum = 0
    tmpSet = set()
    for i in list(gu.nodes):
        for j in list(gl.nodes):
            if (i, j) in list(gc.edges) or (j, i) in list(gc.edges):
                sum += float(gc.edges[i, j]['weight'])
                tmpSet.add(i)
                tmpSet.add(j)
    avgDegree = sum / len(tmpSet)
    if avgDegree > ResultDegree:
        ResultDegree = avgDegree
        ResultSet = tmpSet


if __name__ == '__main__':
    # 定义变量
    # maxGU为GU最大度 userCBCC为用户图中生成的CBCC
    maxGU = 0
    maxGL = 0
    maxGUCBCC = 0
    maxGLCBCC = 0
    # 输入
    m = int(input("度阈值m= "))
    r = int(input("距离阈值r= "))
    ustr = input("查询用户节点（请用英文逗号间隔）：")
    lstr = input("查询位置节点（请用英文逗号间隔）：")
    tmp = ustr.split(",")
    count = len(tmp[0])
    if count != 0:
        for x in tmp:
            GUQuerySet.add('U' + x)

    tmp = lstr.split(",")
    count = len(tmp[0])
    if count != 0:
        for x in tmp:
            GLQuerySet.add('L' + x)

    # 读取文件生成GU GL GC
    load_GU()
    load_GC()
    load_GL()
    print(GU.number_of_nodes())
    print(GU.number_of_edges())
    print(GL.number_of_nodes())
    print(GL.number_of_edges())
    print(GC.number_of_nodes())
    print(GC.number_of_edges())

    start = timeit.default_timer()
    # 算GU和GL的度
    for i in list(GU.nodes):
        tmp = GU.degree[i]
        if tmp > maxGU:
            maxGU = tmp
    for i in list(GL.nodes):
        tmp = GL.degree[i]
        if tmp > maxGL:
            maxGL = tmp

    for x in range(m, maxGU+1):
        if get_CBCC(x, GU, 1):
            maxGUCBCC = x
            exec("GU{} = nx.Graph()".format(x))
            exec("GU{}.add_nodes_from({})".format(x, list(GU.nodes)))
            exec("GU{}.add_edges_from({})".format(x, list(GU.edges)))
            exec("print('GU{}')".format(x))
    for x in range(m, maxGL+1):
        if get_CBCC(x, GL, 0):
            maxGLCBCC = x
            exec("GL{} = nx.Graph()".format(x))
            exec("GL{}.add_nodes_from({})".format(x, list(GL.nodes)))
            exec("GL{}.add_edges_from({})".format(x, list(GL.edges)))
            exec("print('GL{}')".format(x))

    for i in range(m, maxGUCBCC+1):
        for j in range(m, maxGLCBCC+1):
            exec("get_Checkin(GU{}, GL{}, GC)".format(i, j))
            exec("print('GU{} and GL{}')".format(i, j))

    end = timeit.default_timer()
    if ResultSet:
        print("平均签入密度：", ResultDegree)
        print("结果集：", ResultSet)
        print("运行时间：{}秒".format(end-start))
        with open('/content/drive/My Drive/LBSN/Result/ResultList1.txt', 'w') as f:
            for x in ResultSet:
                f.write(x + ',')
            f.write("\n")
            f.write("checkin:")
            f.write(str(ResultDegree))
            f.write("\n")
            f.write("scale:")
            f.write(str(len(ResultSet)))
            f.write("\n")
            f.write("time:")
            f.write(str(end-start))
    else:
        print("结果集为空")
        print("运行时间：{}秒".format(end - start))



度阈值m= 10
距离阈值r= 70
查询用户节点（请用英文逗号间隔）：32
查询位置节点（请用英文逗号间隔）：2
200
2211
100
1287
300
2149
GU10
GU11
GU12
GU13
GU14
GU15
GU16
GL10
GL11
GL12
GL13
GL14
GL15
GL16
GL17
GL18
GU10 and GL10
GU10 and GL11


KeyboardInterrupt: ignored